<a href="https://colab.research.google.com/github/renatojmf/Aspects-Extraction-in-Portuguese/blob/main/CAt_Paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pré-Processamento

In [1]:
! pip install stanza

     |████████████████████████████████| 432 kB 5.6 MB/s 
     |████████████████████████████████| 175 kB 22.5 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=4b1378d6074e4a8ef9b6bfc2e2fcf303d3c807dec30d22dae48d9bf4f319fde2
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


In [ ]:
! pip install spacy_conll

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving reli.csv to reli.csv


In [ ]:
import stanza as st
import pandas as pd
st.download('pt') # download pt model
nlp = st.Pipeline('pt') # initialize pt neural pipeline

In [51]:
! pip install unidecode

     |████████████████████████████████| 235 kB 7.5 MB/s 


In [55]:
df = pd.read_csv('/content/restaurantes.csv',sep=';')

#passar apenas os textos/frases para a função
def getCoNLLU(df):
  dicts = []
  for texto in df:
    doc = nlp(texto) # doc is class Document
    dicts.append(doc.to_dict())

  return dicts

In [56]:
from unidecode import unidecode
import re

words = []

for word in df['text']:
  old_string = unidecode(word)
  new_string = re.sub(r"[^a-zA-Z0-9]"," ",old_string)
  words.append(new_string.lower())

In [58]:
df['text'] = words

In [59]:
df.head()

,text,aspects,aspectsClass
0,hamburgueria artesanal com tempero diferenciad...,tempero,Food
1,excelente peixe para quem esta localizado no ...,peixe,Food
2,serve lanches e cerveja gelada tem mesinhas f...,cerveja,Drinks
3,muito bom o atendimento,atendimento,Service
4,cardapio variado,Cardápio,Food


In [60]:
dicts = getCoNLLU(df['text'])

saida = ''
for sentenca in dicts[0:2]:
  for tokens in sentenca:
    for tks in tokens:
      for tag in tks:
        #print("tag:" ,tag)
        saida += str(tks[tag])  + "\t"
      saida +="\n"
    saida +="\n"
  saida +="\n"


print(saida)

/usr/local/lib/python3.7/dist-packages/stanza/models/common/beam.py:86: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  prevK = bestScoresId // numWords


1	hamburgueria	hamburgueria	NOUN	Gender=Fem|Number=Sing	11	nsubj	0	12	
2	artesanal	artesanal	ADJ	Gender=Fem|Number=Sing	1	amod	13	22	
3	com	com	ADP	4	case	23	26	
4	tempero	tempero	NOUN	Gender=Masc|Number=Sing	1	nmod	27	34	
5	diferenciado	diferenciar	VERB	Gender=Masc|Number=Sing|VerbForm=Part	4	acl	35	47	
6	de	de	ADP	8	case	49	51	
7	primeira	primeiro	ADJ	Gender=Fem|NumType=Ord|Number=Sing	8	amod	52	60	
8	qualidade	qualidade	NOUN	Gender=Fem|Number=Sing	4	nmod	61	70	
9	fora	ser	AUX	Mood=Ind|Number=Sing|Person=3|Tense=Pqp|VerbForm=Fin	11	cop	72	76	
10	o	o	DET	Definite=Def|Gender=Masc|Number=Sing|PronType=Art	11	det	77	78	
11	atendimento	atendimento	NOUN	Gender=Masc|Number=Sing	0	root	79	90	
(12, 13)	no	91	93	
12	em	em	ADP	14	case	
13	o	o	DET	Gender=Masc|Number=Sing|PronType=Art	14	det	
14	qual	qual	PRON	Gender=Masc|Number=Sing|PronType=Rel	15	obl	94	98	
15	diferencia	diferenciar	VERB	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin	11	acl:relcl	99	109	
16	por	por	SCONJ	18	mark	110	113

In [62]:
def getNouns():
  for sentenca in dicts:
    for tokens in sentenca:
      for tks in tokens:
        for tag in tks:
          if tks[tag] == 'NOUN':
            yield (tks['text'].lower())
          

In [63]:
from collections import Counter, defaultdict
import json

c = Counter()
c.update(getNouns())
nouns = getNouns()
print(c)

json.dump(dict(c), open('/content/nouns.json', 'w'))

Counter({'atendimento': 74, 'comida': 52, 'ambiente': 38, 'lugar': 33, 'preco': 31, 'pratos': 23, 'restaurante': 23, 'pena': 16, 'local': 15, 'carne': 15, 'qualidade': 14, 'opcao': 14, 'pessoas': 13, 'servico': 12, 'variedade': 12, 'rodizio': 11, 'visita': 9, 'conta': 9, 'peixe': 8, 'casa': 8, 'cardapio': 8, 'massa': 8, 'carnes': 8, 'almoco': 7, 'prato': 7, 'tambaqui': 7, 'garcons': 7, 'estacionamento': 7, 'tempero': 6, 'reais': 6, 'musica': 6, 'manaus': 6, 'bebidas': 6, 'problema': 6, 'localizacao': 6, 'pizza': 5, 'garcom': 5, 'regiao': 5, 'peixes': 5, 'sucos': 5, 'buffet': 5, 'carte': 5, 'delicia': 5, 'valor': 5, 'precos': 5, 'cervejas': 4, 'vezes': 4, 'pizzas': 4, 'acompanhamento': 4, 'arroz': 4, 'vivo': 4, 'camarao': 4, 'entrada': 4, 'resultado': 4, 'batida': 4, 'cidade': 4, 'experiencia': 4, 'acustica': 4, 'jantar': 4, 'mundo': 4, 'padrao': 4, 'pais': 4, 'salgados': 4, 'opcoes': 4, 'empanadas': 3, 'vinho': 3, 'decoracao': 3, 'beira': 3, 'cozida': 3, 'brocolis': 3, 'coco': 3, 'cupu

In [64]:
def getCorpus2(df):
  corpus = []
  for item in df:
    corpus.append(item.split())
  
  return corpus

In [65]:
from gensim.models import Word2Vec
corpus = getCorpus2(df['text'])
#print(corpus[1::])

f = Word2Vec(corpus[1::],
                 sg=0,
                 negative=5,
                 window=10,
                 size=200,
                 min_count=2,
                 iter=5,
                 workers=10)

f.wv.save_word2vec_format("/content/my_word_vectors.vec")


In [66]:
d = json.load(open('/content/nouns.json'))
nouns2 = Counter()
#print(f.wv.vocab.items[0])
for k, v in d.items():
  for tag,_ in f.wv.vocab.items():
    if k.lower() == tag:
      #print("valor v: ",v)
      nouns2[k.lower()] += v

# nouns2, _ = zip(*sorted(nouns2.items(),
#                 key=lambda x: x[1],
#                 reverse=True))

# json.dump(nouns, open("/content/aspect_words.json", "w"))

In [67]:
nouns3, _ = zip(*sorted(nouns2.items(),
                key=lambda x: x[1],
                reverse=True))

json.dump(nouns3, open("/content/aspect_words.json", "w"))

# Execução

In [ ]:
! pip install reach

In [68]:
import json
from reach import Reach
from collections import defaultdict
import numpy as np
from collections import defaultdict
from sklearn.metrics.pairwise import rbf_kernel
from collections import Counter

In [69]:
def normalize(x):
    """Normalize a vector while controlling for zero vectors."""
    x = np.copy(x)
    if np.ndim(x) == 1:
        norm = np.linalg.norm(x)
        if norm == 0:
            return x
        return x / np.linalg.norm(x)
    norm = np.linalg.norm(x, axis=-1)
    mask = norm > 0
    x[mask] /= norm[mask][:, None]
    return x


In [70]:
"""Simple method."""
import numpy as np
from collections import defaultdict
from sklearn.metrics.pairwise import rbf_kernel
from collections import Counter


def get_aspects(fragments, embeddings, n_adj_seed, n_nouns, min_count):
    """Get aspects based on fragments."""
    adj, _, noun = zip(*fragments)
    adj_cand, _ = zip(*Counter(adj).most_common(n_adj_seed))

    cands = candidate(embeddings,
                      adj,
                      noun,
                      adj_cand,
                      n_nouns,
                      min_count)

    return cands


def candidate(embeddings,
              adj,
              noun,
              seed_words,
              n_nouns,
              min_count):
    """
    Generates candidate aspects based on adjective co-occurrences

    Parameters
    ----------
    embeddings : Reach
        A Reach instance containing the word embeddings.
    constructions : list of tuples
        A list of adjective noun tuples.
    seed_words : list of str
        A list of strings. All these words should be in vocab for the
        given embeddings model.
    frequency_threshold : int
        Any noun occurring fewer times than this threshold is discarded
    n_nouns : int
        The amount of items to return

    Returns
    -------
    candidates : dict
        A dictionary mapping strings to their scores.

    """
    a = list(set(adj))
    sims = embeddings.similarity(a, seed_words).max(1)
    adj_scores = dict(zip(a, sims))

    noun_scores = defaultdict(lambda: [0, 0])
    for adj, noun in zip(adj, noun):
        noun_scores[noun][0] += adj_scores[adj]
        noun_scores[noun][1] += 1

    noun_scores = {k: v[0] for k, v in noun_scores.items()
                   if v[1] > min_count}

    return sorted(noun_scores.items(), key=lambda x: x[1])[-n_nouns:]


def rbf_attention(vec, memory, gamma, **kwargs):
    """
    Single-head attention using RBF kernel.

    Parameters
    ----------
    vec : np.array
        an (N, D)-shaped array, representing the tokens of an instance.
    memory : np.array
        an (M, D)-shaped array, representing the memory items
    gamma : float
        the gamma of the RBF kernel.

    Returns
    -------
    attention : np.array
        A (1, N)-shaped array, representing a single-headed attention mechanism

    """
    z = rbf_kernel(vec, memory, gamma)
    s = z.sum()
    if s == 0:
        # If s happens to be 0, back off to uniform
        return np.ones((1, len(vec))) / len(vec)
    return (z.sum(1) / s)[None, :]


def softmax(x, axis=1):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x, axis, keepdims=True))
    s = e_x.sum(axis=axis, keepdims=True)
    return e_x / s


def attention(vec, memory, **kwargs):
    """
    Standard multi-head attention mechanism.

    Parameters
    ----------
    vec : np.array
        an (N, D)-shaped array, representing the tokens of an instance.
    memory : np.array
        an (M, D)-shaped array, representing the memory items

    Returns
    -------
    attention : np.array
        A (M, N)-shaped array, representing the attention over all memories.

    """
    z = memory.dot(vec.T)
    return softmax(z)


def mean(vec, aspect_vecs, **kwargs):
    """Just a mean weighting."""
    return (np.ones(len(vec)) / len(vec))[None, :]





In [104]:
def get_scores(instances,
               aspects,
               r,
               labels,
               remove_oov=False,
               attention_func=attention,
               **kwargs):
    """Scoring function."""
    assert all([x in r.items for x in labels])
    
    label_vecs = normalize(r.vectorize(labels))
    aspect_vecs = [x.mean(0)
                   for x in r.transform(aspects,
                                        remove_oov=False)]
    aspect_vecs = np.stack(aspect_vecs)
    if len(instances) == 1:
        instances = [instances]

    t = r.transform(instances, remove_oov=remove_oov)

    out = []
    for vec in t:
        att = attention_func(vec, aspect_vecs, **kwargs)
        # Att = (n_heads, n_words)
        z = att.dot(vec)
        # z = (n_heads, n_dim)
        x = normalize(z).dot(label_vecs.T)
        # x = (n_heads, n_labels)
        out.append(x.sum(0))
    return np.stack(out)

In [131]:
GAMMA = .03
N_ASPECT_WORDS = 200

scores = defaultdict(dict)
r = Reach.load("/content/my_word_vectors.vec",unk_word="<UNK>")

aspects = [[x] for x in json.load(open("/content/aspect_words.json"))]
aspects = aspects[:N_ASPECT_WORDS]

#atendimento fantástico, voltarei mais vezes.".split(), "atendimento fantástico, voltarei mais vezes.".split(),"comida bem temperada como deve ser.".split()
# instances = ["atendimento fantástico, garçons educados, voltarei mais vezes".split(), 
#              "Atendimento sempre excelente !".split(), 
#              "comida bem temperada como deve ser.".split()]
instances = df['text'][0:3]
label_set = {"atendimento", "comida", "ambiente"}
#[i.split() for i in instances]
s = get_scores([i.split() for i in instances],
               aspects,
               r,
               label_set,
               gamma=GAMMA,
               remove_oov=False,
               attention_func=rbf_attention)

pred = s.argmax(1)
print(s)
print(pred)

[[0.94458807 0.93739843 0.95991635]
 [0.93514323 0.9300681  0.94181776]
 [0.94294    0.92752093 0.94879276]]
[2 2 2]


In [110]:
BEST_ATT = {"n_noun": 980}
BEST_RBF = {"n_noun": 200}

scores = defaultdict(dict)
r = Reach.load("/content/my_word_vectors.vec",
                   unk_word="<UNK>")

d = json.load(open("/content/nouns.json"))

nouns = Counter()
for k, v in d.items():
  if k.lower() in r.items:
    nouns[k.lower()] += v

embedding_paths = ["/content/my_word_vectors.vec"]
bundles = ((rbf_attention, attention), embedding_paths)

In [112]:
from itertools import product
for att, path in product(*bundles):
  r = Reach.load(path, unk_word="<UNK>")

  if att == rbf_attention:
    candidates, _ = zip(*nouns.most_common(BEST_RBF["n_noun"]))
  else:
    candidates, _ = zip(*nouns.most_common(BEST_ATT["n_noun"]))

  aspects = [[x] for x in candidates]

In [136]:
df['aspects'].unique()

array(['tempero', 'peixe', 'cerveja', 'atendimento', 'Cardápio',
       'Ambiente', ' Custo-benefício', ' atendimento', ' ambiente',
       ' vista', 'vinho', 'empanadas', 'cervejas', 'lugar', 'pratos',
       'comida', 'serviço', 'Comida', 'preço', 'pizza', 'almoço',
       'Delivery', 'horário', 'restaurante', 'Lugar', 'Serviço', 'Som',
       'Coco bambu', 'sobremesa a cocada ao forno com sorvete', 'garçom',
       'cardápio', 'local', 'música ao vivo', 'sorveteria',
       'sorvete de Creme de Cupuaçu', 'pessoal', 'Food', 'peixaria',
       'culinária', 'Restaurante Banzeiro', 'criatividade',
       'sabores regionais', 'Peixe', 'acompanhamentos', 'ambiente',
       'Atendimento', 'sucos', 'batida de cacau', 'tambaqui na brasa',
       'garçons', 'garçom Ebrahim', 'Comida regional', 'peixarias',
       'buffet', 'Preço', 'açaí', 'opção', 'preço cobrado', 'carnes',
       'Pratos quentes', 'comida japonesa', 'peixes', 'localização',
       'Casa', 'Buffet', 'Churrascaria', 'tempo de

# Funções auxiliares

In [ ]:
with open('/content/restaurant_textos.txt ', 'w') as textfile:
  textfile.write("\n".join(str(item) for item in sentences))
textfile.close()

In [ ]:
with open('/content/corpus.txt ', 'w') as textfile:
  for item in corpus:
    textfile.write(str(item))
textfile.close()